1. 집합과 원소
- 집합 : 객관적으로 범위를 규정한 무언가
- 원소 : 집합 내의 각각의 어떤 것
2. 부분집합, 진 부분집합
- 부분 집합 : A의 모든 원소가 B의 원소일때
- 진부분 집합 : 집합 A의 모든 원사고 B의 원소이며 A와 집합 B가 같지않은 집합
3. 집합의 연산
- 합집합 : A와 B 적어도 한쪽에 속하는 집합 (A or B)
- 교집합 : A와 B에 모두 속하는 집합(A and B)
- 차집합 : A원소 중 B원소를 제외한 원소의 집합(A and ~B)

배열로 집합만들기

In [ ]:
/* int형 집합 IntSet(헤더 부분) */
#ifndef ___IntSet
#define ___IntSet

/*--- int형 집합을 관리하는 구조체 ---*/
typedef struct {
	int max;	/* 집합의 크기 */
	int num;	/* 집합의 원소 개수 */
	int *set;	/* 집합 본체의 배열(의 첫 번째 요소에 대한 포인터) */
} IntSet;

/*--- 집합 초기화 ---*/
int Initialize(IntSet *s, int max);

/*--- 집합 s에 n이 들어있는지 확인 ---*/
int IsMember(const IntSet *s, int n);

/*--- 집합 s에 n을 추가 ---*/
void Add(IntSet *s, int n);

/*--- 집합 s에서 n을 삭제 ---*/
void Remove(IntSet *s, int n);

/*--- 집합 s에 넣을 수 있는 최대 원소 개수를 반환 ---*/
int Capacity(const IntSet *s);

/*--- 집합 s의 원소 개수 ---*/
int Size(const IntSet *s);

/*--- 집합 s2를 s1에 대입 ---*/
void Assign(IntSet *s1, const IntSet *s2);

/*--- 집합 s1과 s2가 같은지 확인 ---*/
int Equal(const IntSet *s1, const IntSet *s2);

/*--- 집합 s2와 s3의 합집합을 s1에 대입 ---*/
IntSet *Union(IntSet *s1, const IntSet *s2, const IntSet *s3);

/*--- 집합 s2와 s3의 교집합을 s1에 대입 ---*/
IntSet *Intersection(IntSet *s1, const IntSet *s2, const IntSet *s3);

/*--- 집합 s2에서 s3을 뺀 차집합을 s1에 대입 ---*/
IntSet *Difference(IntSet *s1, const IntSet *s2, const IntSet *s3);

/*--- 집합 s의 모든 원소를 ​​출력 ---*/
void Print(const IntSet *s);

/*--- 집합 s의 모든 원소를 ​​출력(줄 바꿈 문자 포함) ---*/
void PrintLn(const IntSet *s);

/*--- 집합을 메모리에서 제거 ---*/
void Terminate(IntSet *s);
#endif

In [ ]:
/* int형 집합 IntSet(소스 부분) */
#include <stdio.h>
#include <stdlib.h>
#include "IntSet.h"

/*--- 집합 초기화 ---*/
int Initialize(IntSet *s, int max)
{
	s->num = 0;
	if ((s->set = calloc(max, sizeof(int))) == NULL) {
		s->max = 0;	/* 배열 생성에 실패 */
		return -1;
	}
	s->max = max;
	return 0;
}

/*--- 집합 s에 n이 들어있는지 확인 ---*/
int IsMember(const IntSet *s, int n)
{
	int i;
	for (i = 0; i < s->num; i++)
		if (s->set[i] == n)
			return i; 	/* 들어있음(인덱스를 반환) */
	return -1; 			/* 들어있지 않음 */
}

/*--- 집합 s에 n을 추가 ---*/
void Add(IntSet *s, int n)
{
	if (s->num < s->max && IsMember(s, n) == -1) 	/* 들어있지 않으면 */
		s->set[s->num++] = n; 						/* 배열 끝에 n을 추가 */
}

/*--- 집합 s에서 n을 삭제 ---*/
void Remove(IntSet *s, int n)
{
	int idx;
	if ((idx = IsMember(s, n)) != -1) {
		s->set[idx] = s->set[--s->num];				/* 마지막 요소를 삭제 위치로 옮김 */
	}
}

/*--- 집합 s에 넣을 수 있는 최대 원소 개수를 반환 ---*/
int Capacity(const IntSet *s)
{
	return s->max;
}

/*--- 집합 s의 원소 개수를 반환 ---*/
int Size(const IntSet *s)
{
	return s->num;
}

/*--- 집합 s2를 s1에 대입 ---*/
void Assign(IntSet *s1, const IntSet *s2)
{
	int i;
	int n = (s1->max < s2->num) ? s1->max : s2->num;	/* 복사하는 원소의 개수 */
	for (i = 0; i < n; i++)
		s1->set[i] = s2->set[i];
	s1->num = n;
}

/*--- 집합 s1과 s2가 같은지 확인 ---*/
int Equal(const IntSet *s1, const IntSet *s2)
{
	int i, j;
	if (Size(s1) != Size(s2))
		return 0;
	for (i = 0; i < s1->num; i++) {
		for (j = 0; j < s2->num; j++)
			if (s1->set[i] == s2->set[j])
				break;
		if (j == s2->num)
			return 0;
	}
	return 1;
}

/*--- 집합 s2와 s3의 합집합을 s1에 대입 ---*/
IntSet *Union(IntSet *s1, const IntSet *s2, const IntSet *s3)
{
	int i;
	Assign(s1, s2);
	for (i = 0; i < s3->num; i++)
		Add(s1, s3->set[i]);
	return s1;
}

/*--- 집합 s2와 s3의 교집합을 s1에 대입 ---*/
IntSet *Intersection(IntSet *s1, const IntSet *s2, const IntSet *s3)
{
	int i, j;
	s1->num = 0;		/* s1을 공집합으로 만듭니다. */
	for (i = 0; i < s2->num; i++)
		for (j = 0; j < s3->num; j++)
			if (s2->set[i] == s3->set[j])
				Add(s1, s2->set[i]);
	return s1;
}

/*--- 집합 s2에서 s3를 뺀 차집합을 s1에 대입 ---*/
IntSet *Difference(IntSet *s1, const IntSet *s2, const IntSet *s3)
{
	int i, j;
	s1->num = 0;		/* s1을 공집합으로 만듭니다. */
	for (i = 0; i < s2->num; i++) {
		for (j = 0; j < s3->num; j++)
			if (s2->set[i] == s3->set[j])
				break;
		if (j == s3->num)
			Add(s1, s2->set[i]);
	}
	return s1;
}

/*--- 집합 s의 모든 원소를 ​​출력 ---*/
void Print(const IntSet *s)
{
	int i;

	printf("{ ");
	for (i = 0; i < s->num; i++)
		printf("%d ", s->set[i]);
	printf("}");
}

/*--- 집합 s의 모든 원소를 출력(줄 바꿈 문자 포함) ---*/
void PrintLn(const IntSet *s)
{
	Print(s);
	putchar('\n');
}

/*--- 집합 종료 ---*/
void Terminate(IntSet *s)
{
	if (s->set != NULL) {
		free(s->set);	/* 배열 해제 */
		s->max = s->num = 0;
	}
}

In [ ]:
/* int형 집합 IntSet의 사용 예(1) */
#include <stdio.h>
#include "IntSet.h"

int main(void)
{
	IntSet s1, s2, s3;
	Initialize(&s1, 24);
	Initialize(&s2, 24);
	Initialize(&s3, 24);

	Add(&s1, 10);			/* s1 = {10} */
	Add(&s1, 15);			/* s1 = {10, 15} */
	Add(&s1, 20);			/* s1 = {10, 15, 20} */
	Add(&s1, 25);			/* s1 = {10, 15, 20, 25} */

	Assign(&s2, &s1);		/* s2 = {10, 15, 20, 25} */
	Add(&s2, 12);			/* s2 = {10, 15, 20, 25, 12} */
	Remove(&s2, 20);		/* s2 = {10, 15, 12, 25} */
	Assign(&s3, &s2);		/* s3 = {10, 15, 12, 25} */

	printf("s1 = "); PrintLn(&s1);
	printf("s2 = "); PrintLn(&s2);
	printf("s3 = "); PrintLn(&s3);

	printf("집합 s1에 15가 들어있%s.\n", IsMember(&s1, 15) > 0 ? "습니다" : "지 않습니다");
	printf("집합 s2에 25가 들어있%s.\n", IsMember(&s2, 25) > 0 ? "습니다" : "지 않습니다");
	printf("집합 s1과 s2는 서로 같%s.\n", Equal(&s1, &s2) ? "습니다" : "지 않습니다.");
	printf("집합 s2와 s3는 서로 같%s.\n", Equal(&s2, &s3) ? "습니다" : "지 않습니다.");

	Terminate(&s1);
	Terminate(&s2);
	Terminate(&s3);

	return 0;
}

In [ ]:
/* int형 집합 IntSet의 사용 예(2) */
#include <stdio.h>
#include "IntSet.h"

enum { ADD, RMV, SCH };

/*--- 데이터 입력 ---*/
int scan_data(int sw)
{
	int data;
	
	switch (sw) {
	case ADD: printf("추가할 데이터 : "); break;
	case RMV: printf("삭제할 데이터 : "); break;
	case SCH: printf("검색할 데이터 : "); break;
	}

	scanf("%d", &data);

	return data;
}

int main(void)
{
	IntSet s1, s2, temp;
	
	Initialize(&s1, 512); 
	Initialize(&s2, 512); 
	Initialize(&temp, 512);

	while (1) {
		int m, x, idx;

		printf("s1 = "); PrintLn(&s1);
		printf("s2 = "); PrintLn(&s2);
		printf("(1) s1에 추가 (2) s1에서 삭제 (3) s1에서 검색 (4) s1←s2 (5) 여러 연산\n",
			"(6) s2에 추가 (7) s2에서 삭제 (8) s2에서 검색 (9) s2←s1 (0) 종료 : ");
		scanf("%d", &m);

		if (m == 0) break;
		switch (m) {
		case 1: x = scan_data(ADD); Add(&s1, x); break;			/* 추가 */
		case 2: x = scan_data(RMV); Remove(&s1, x); break;		/* 삭제 */
		case 3: x = scan_data(SCH); idx = IsMember(&s1, x);		/* 검색 */
			printf("s1에 들어있%s.\n", idx >= 0 ? "습니다" : "지 않습니다"); break;
		case 4: Assign(&s1, &s2); break;						/* s2를 s1에 대입 */
		case 5: printf("s1 == s2 = %s\n", Equal(&s1, &s2) ? "true" : "false");
			printf("s1 & s2 = "); Intersection(&temp, &s1, &s2);
			PrintLn(&temp);
			printf("s1 | s2 = "); Union(&temp, &s1, &s2);
			PrintLn(&temp);
			printf("s1 - s2 = "); Difference(&temp, &s1, &s2);
			PrintLn(&temp);
			break;
		case 6: x = scan_data(ADD); Add(&s2, x); break;			/* 추가 */
		case 7: x = scan_data(RMV); Remove(&s2, x); break;		/* 삭제 */
		case 8: x = scan_data(SCH); idx = IsMember(&s2, x);		/* 검색 */
			printf("s2에 들어있%s.\n", idx >= 0 ? "습니다" : "지 않습니다"); break;
		case 9: Assign(&s2, &s1); break;						/* s1을 s2에 대입 */
		}
	}

	Terminate(&s1); 
	Terminate(&s2); 
	Terminate(&temp);
	
	return 0;
}

In [ ]:
/* Q1 실습 7-2 프로그램(IntSet.c)에 8개 함수 추가 */
#include <stdio.h>
#include <stdlib.h>
#include "IntSet.h"

/*--- ■ 표시가 있는 함수가 새로 추가된 8개의 함수 입니다. ---*/

int Initialize(IntSet *s, int max)
{
	s->num = 0;
	if ((s->set = calloc(max, sizeof(int))) == NULL) {
		s->max = 0;								
		return -1;
	}
	s->max = max;
	return 0;
}

int IsMember(const IntSet *s, int n)
{
	int i;

	for (i = 0; i < s->num; i++)
		if (s->set[i] == n)
			return i;			
	return -1;					
}

/*--- ■ 집합이 가득 찼다면 1, 아니면 0을 반환 ---*/
int IsFull(const IntSet *s)
{
	return s->num >= s->max;
}

void Add(IntSet *s, int n)
{
	if (s->num < s->max && IsMember(s, n) == -1)	
		s->set[s->num++] = n;						
}

void Remove(IntSet *s, int n)
{
	int idx;

	if ((idx = IsMember(s, n)) != -1) {
		s->set[idx] = s->set[--s->num];	
	}
}

int Capacity(const IntSet *s)
{
	return s->max;
}

int Size(const IntSet *s)
{
	return s->num;
}

void Assign(IntSet *s1, const IntSet *s2)
{
	int i;
	int n = (s1->max < s2->num) ? s1->max : s2->num;	

	for (i = 0; i < n; i++)
		s1->set[i] = s2->set[i];
	s1->num = n;
}

int Equal(const IntSet *s1, const IntSet *s2)
{
	int i, j;

	if (Size(s1) != Size(s2))
		return 0;

	for (i = 0; i < s1->num; i++) {
		for (j = 0; j < s2->num; j++)
			if (s1->set[i] == s2->set[j])
				break;
		if (j == s2->num)
			return 0;
	}
	return 1;
}

IntSet *Union(IntSet *s1, const IntSet *s2, const IntSet *s3)
{
	int i;

	Assign(s1, s2);
	for (i = 0; i < s3->num; i++)
		Add(s1, s3->set[i]);

	return s1;
}

IntSet *Intersection(IntSet *s1, const IntSet *s2, const IntSet *s3)
{
	int i, j;

	s1->num = 0;			/* s1を空集合にする */
	for (i = 0; i < s2->num; i++)
		for (j = 0; j < s3->num; j++)
			if (s2->set[i] == s3->set[j])
				Add(s1, s2->set[i]);
	return s1;
}

IntSet *Difference(IntSet *s1, const IntSet *s2, const IntSet *s3)
{
	int i, j;

	s1->num = 0;			/* s1を空集合にする */
	for (i = 0; i < s2->num; i++) {
		for (j = 0; j < s3->num; j++)
			if (s2->set[i] == s3->set[j])
				break;
		if (j == s3->num)
			Add(s1, s2->set[i]);
	}
	return s1;
}

/*--- ■ 집합 s2, s3의 대칭 차를 s1에 대입하는 함수 ---*/
IntSet *SymmetricDifference(IntSet *s1, const IntSet *s2, const IntSet *s3)
{
	int i;

	s1->num = 0;			/* s1을 공집합으로 */

	for (i = 0; i < s2->num; i++)
		if (IsMember(s3, s2->set[i]) == -1)
			Add(s1, s2->set[i]);

	for (i = 0; i < s3->num; i++)
		if (IsMember(s2, s3->set[i]) == -1)
			Add(s1, s3->set[i]);

	return s1;
}

/*--- ■ 집합 s1에 s2의 모든 원소를 추가하는 함수(s1 포인터 반환) ---*/
IntSet *ToUnion(IntSet *s1, const IntSet *s2)
{
	int i;

	for (i = 0; i < s2->num; i++)
		Add(s1, s2->set[i]);

	return s1;
}

/*--- ■ 집합 s1에서 s2에 들어 있지 않은 모든 원소를 삭제하는 함수(s1 포인터 반환) ---*/
IntSet *ToIntersection(IntSet *s1, const IntSet *s2)
{
	int i = 0;

	while (i < s1->num) {
		if (IsMember(s2, s1->set[i]) == -1)
			Remove(s1, s1->set[i]);
		else
			i++;
	}
	return s1;
}

/*--- ■ 집합 s1에서 s2에 들어 있는 모든 원소를 삭제하는 함수(s1 포인터 반환) ---*/
IntSet *ToDifference(IntSet *s1, const IntSet *s2)
{
	int i;

	for (i = 0; i < s2->num; i++)
		Remove(s1, s2->set[i]);

	return s1;
}

/*--- ■ 집합 s1이 s2의 부분집합이면 1, 아니면 0을 반환 ---*/
int IsSubset(const IntSet *s1, const IntSet *s2)
{
	int i, j;

	for (i = 0; i < s1->num; i++) {
		for (j = 0; j < s2->num; j++)
			if (s1->set[i] == s2->set[j])
				break;
		if (j == s2->num)					
			return 0;
	}
	return 1;
}

/*--- ■ 집합 s1이 s2의 진부분집합이면 1, 아니면 0을 반환 ---*/
int IsProperSubset(const IntSet *s1, const IntSet *s2)
{
	int i;

	if (s1->num >= s2->num)					
		return 0;							

	return IsSubset(s1, s2);
}

void Print(const IntSet *s)
{
	int i;

	printf("{ ");
	for (i = 0; i < s->num; i++)
		printf("%d ", s->set[i]);
	printf("}");
}

void PrintLn(const IntSet *s)
{
	Print(s);
	putchar('\n');
}

/*--- ■ 집합의 모든 원소를 삭제하는 함수 ---*/
void Clear(IntSet *s)
{
	s->num = 0;
}

void Terminate(IntSet *s)
{
	if (s->set != NULL) {
		free(s->set);							
		s->max = s->num = 0;
	}
}


In [ ]:
/* Q2 배열의 원소가 오름차순을 유지하는 int형 SortedIntSet*/
#include <stdio.h>
#include <stdlib.h>
#include "SortedIntSet.h"

int Initialize(SortedIntSet *s, int max)
{
	s->num = 0;
	if ((s->set = calloc(max, sizeof(int))) == NULL) {
		s->max = 0;								
		return -1;
	}
	s->max = max;
	return 0;
}

int _search(const SortedIntSet *s, int n, int *flag)
{
	int pl = 0;				
	int pr = s->num - 1;	
	int pc;					

	*flag = 1;
	if (s->num <= 0) return 0;		

	do {
		pc = (pl + pr) / 2;
		if (s->set[pc] == n) {		
			*flag = 0;
			return pc;
		}
		else if (s->set[pc] < n)
			pl = pc + 1;
		else
			pr = pc - 1;
	} while (pl <= pr);

	return pl;						
}

int IsMember(const SortedIntSet *s, int n)
{
	int flag;
	int idx = _search(s, n, &flag);
	return flag ? idx : -1;
}

int IsFull(const SortedIntSet *s)
{
	return s->num >= s->max;
}

void Add(SortedIntSet *s, int n)
{
	int i, idx, flag;

	if (s->num < s->max) {
		idx = _search(s, n, &flag);
		if (flag == 1) {					
			s->num++;
			for (i = s->num - 1; i > idx; i--)
				s->set[i] = s->set[i - 1];
			s->set[idx] = n;					
		}
	}
}

void Remove(SortedIntSet *s, int n)
{
	int i, idx, flag;

	if (s->num > 0) {
		idx = _search(s, n, &flag);
		if (flag == 0) {						
			--s->num;
			for (i = idx; i < s->num; i++)		
				s->set[i] = s->set[i + 1];		
		}
	}
}

int Capacity(const SortedIntSet *s)
{
	return s->max;
}

int Size(const SortedIntSet *s)
{
	return s->num;
}

void Assign(SortedIntSet *s1, const SortedIntSet *s2)
{
	int i;
	int n = (s1->max < s2->num) ? s1->max : s2->num;	

	for (i = 0; i < n; i++)
		s1->set[i] = s2->set[i];
	s1->num = n;
}

int Equal(const SortedIntSet *s1, const SortedIntSet *s2)
{
	int i;

	if (Size(s1) != Size(s2))
		return 0;

	for (i = 0; i < s1->num; i++)
		if (s1->set[i] != s2->set[i])
			return 0;
	return 1;
}

SortedIntSet *Union(SortedIntSet *s1, const SortedIntSet *s2, const SortedIntSet *s3)
{
	int k2, k3;

	s1->num = 0;
	k2 = k3 = 0;
	while (k2 < s2->num && k3 < s3->num) {
		if (s2->set[k2] < s3->set[k3])
			s1->set[s1->num++] = s2->set[k2++];
		else if (s2->set[k2] > s3->set[k3])
			s1->set[s1->num++] = s3->set[k3++];
		else {
			s1->set[s1->num++] = s2->set[k2++];
			k3++;
		}
		if (s1->num == s1->max) return s1;
	}

	if (k2 < s2->num)
		while (k2 < s2->num  &&  s1->num < s1->max)
			s1->set[s1->num++] = s2->set[k2++];
	else
		while (k3 < s3->num  &&  s1->num < s1->max)
			s1->set[s1->num++] = s3->set[k3++];

	return s1;
}

SortedIntSet *Intersection(SortedIntSet *s1, const SortedIntSet *s2, const SortedIntSet *s3)
{
	int k2, k3;

	s1->num = 0;
	k2 = k3 = 0;
	while (k2 < s2->num && k3 < s3->num) {
		if (s2->set[k2] < s3->set[k3])
			k2++;
		else if (s2->set[k2] > s3->set[k3])
			k3++;
		else {
			s1->set[s1->num++] = s2->set[k2];
			k3++;
			if (s1->num == s1->max)
				return s1;
		}
	}
	return s1;
}

SortedIntSet *Difference(SortedIntSet *s1, const SortedIntSet *s2, const SortedIntSet *s3)
{
	int k2, k3;

	s1->num = 0;
	k2 = k3 = 0;
	while (k2 < s2->num && k3 < s3->num) {
		if (s2->set[k2] < s3->set[k3])
			s1->set[s1->num++] = s2->set[k2++];
		else if (s2->set[k2] > s3->set[k3])
			k3++;
		else {
			k2++;
			k3++;
		}
		if (s1->num == s1->max) return s1;
	}

	if (k2 < s2->num)
		while (k2 < s2->num  &&  s1->num < s1->max)
			s1->set[s1->num++] = s2->set[k2++];
	return s1;
}

SortedIntSet *SymmetricDifference(SortedIntSet *s1, const SortedIntSet *s2, const SortedIntSet *s3)
{
	int k2, k3;

	s1->num = 0;
	k2 = k3 = 0;
	while (k2 < s2->num && k3 < s3->num) {
		if (s2->set[k2] < s3->set[k3])
			s1->set[s1->num++] = s2->set[k2++];
		else if (s2->set[k2] > s3->set[k3])
			s1->set[s1->num++] = s3->set[k3++];
		else {
			k2++;
			k3++;
		}
		if (s1->num == s1->max) return s1;
	}

	if (k2 < s2->num)
		while (k2 < s2->num && s1->num < s1->max)
			s1->set[s1->num++] = s2->set[k2++];
	else
		while (k3 < s3->num && s1->num < s1->max)
			s1->set[s1->num++] = s3->set[k3++];

	return s1;
}

SortedIntSet *ToUnion(SortedIntSet *s1, const SortedIntSet *s2)
{
	int i;

	for (i = 0; i < s2->num; i++)
		Add(s1, s2->set[i]);

	return s1;
}

SortedIntSet *ToIntersection(SortedIntSet *s1, const SortedIntSet *s2)
{
	int i = 0;

	while (i < s1->num) {
		if (IsMember(s2, s1->set[i]) == -1)
			Remove(s1, s1->set[i]);
		else
			i++;
	}
	return s1;
}

SortedIntSet *ToDifference(SortedIntSet *s1, const SortedIntSet *s2)
{
	int i;

	for (i = 0; i < s2->num; i++)
		Remove(s1, s2->set[i]);

	return s1;
}

int IsSubset(const SortedIntSet *s1, const SortedIntSet *s2)
{
	int i, j;

	for (i = 0; i < s1->num; i++) {
		for (j = 0; j < s2->num; j++)
			if (s1->set[i] == s2->set[j])
				break;
		if (j == s2->num)					
			return 0;
	}
	return 1;
}

int IsProperSubset(const SortedIntSet *s1, const SortedIntSet *s2)
{
	int i;

	if (s1->num >= s2->num)					
		return 0;							

	return IsSubset(s1, s2);
}

void Print(const SortedIntSet *s)
{
	int i;

	printf("{ ");
	for (i = 0; i < s->num; i++)
		printf("%d ", s->set[i]);
	printf("}");
}

void PrintLn(const SortedIntSet *s)
{
	Print(s);
	putchar('\n');
}

void Clear(SortedIntSet *s)
{
	s->num = 0;
}

void Terminate(SortedIntSet *s)
{
	if (s->set != NULL) {
		free(s->set);							
		s->max = s->num = 0;
	}
}


In [ ]:
/* 비트 벡터 집합 BitSet(헤더 부분) */

#ifndef ___BitSet
#define ___BitSet

typedef unsigned long BitSet;			/* 집합을 나타내는 형 */

#define BitSetNull    (BitSet)0			/* 공집합 */
#define BitSetBits     32				/* 유효 비트 수 */
#define SetOf(no)  ((BitSet)1 << (no))	/* 집합 {no} */

/*--- 집합 s에 n이 있는지 확인 ---*/
int IsMember(BitSet s, int n);

/*--- 집합 s에 n을 추가 ---*/
void Add(BitSet *s, int n);

/*--- 집합 s에서 n을 삭제 ---*/
void Remove(BitSet *s, int n);

/*--- 집합 s의 원소 개수를 반환 ---*/
int Size(BitSet s);

/*--- 집합 s의 모든 원소를 ​​출력 ---*/
void Print(BitSet s);

/*--- 집합 s의 모든 원소를 ​​출력(줄 바꿈 문자 포함) ---*/
void PrintLn(BitSet s);

#endif

In [ ]:
/* 비트 벡터에 의한 정수 집합 연산(소스 부분) */
#include <stdio.h>
#include "BitSet.h"

/*--- 집합 s에 n이 들어있는지 확인 ---*/
int IsMember(BitSet s, int n)
{
	return s & SetOf(n);
}

/*--- 집합 s에 n을 추가 ---*/
void Add(BitSet *s, int n)
{
	*s |= SetOf(n);
}

/*--- 집합 s에서 n을 삭제 ---*/
void Remove(BitSet *s, int n)
{
	*s &= ~SetOf(n);
}

/*--- 집합 s의 원소 개수를 반환 ---*/
int Size(BitSet s)
{
	int n = 0;
	for (; s != BitSetNull; n++)
		s &= s - 1;
	return n;
}

/*--- 집합 s의 모든 원소를 ​​출력 ---*/
void Print(BitSet s)
{
	int i;
	printf("{ ");
	for (i = 0; i < BitSetBits; i++)
		if (IsMember(s, i))
			printf("%d ", i);
	printf("}");
}

/*--- 집합 s의 모든 원소를 ​​출력(줄 바꿈 문자 포함) ---*/
void PrintLn(BitSet s)
{
	Print(s);
	putchar('\n');
}

In [ ]:
/* 비트 벡터로 정수 집합 표현 */
#include <stdio.h>
#include "BitSet.h"

enum { ADD, RMV, SCH };

/*--- 데이터 입력 ---*/
int scan_data(int sw)
{
	int data;

	switch (sw) {
	case ADD: printf("추가할 데이터 : "); break;
	case RMV: printf("삭제할 데이터 : "); break;
	case SCH: printf("검색할 데이터 : "); break;
	}

	scanf("%d", &data);

	return data;
}

int main(void)
{
	BitSet s1 = BitSetNull;
	BitSet s2 = BitSetNull;
	
	while (1) {
		int m, x, idx;

		printf("s1 = "); PrintLn(s1);
		printf("s2 = "); PrintLn(s2);
		printf("(1) s1에 추가 (2) s1에서 삭제 (3) s1에서 검색 (4) s1←s2 (5) 여러 연산\n"
			"(6) s2에 추가 (7) s2에서 삭제 (8) s2에서 검색 (9) s2←s1 (0) 종료 : ");
		scanf("%d", &m);
		if (m == 0) break;
		switch (m) {
		case 1: x = scan_data(ADD); Add(&s1, x);	break; 	/* 추가 */
		case 2: x = scan_data(RMV); Remove(&s1, x);	break; 	/* 삭제 */
		case 3: x = scan_data(SCH); idx = IsMember(s1, x);	/* 검색 */
			printf("s1에 들어있%s.\n", idx != 0 ? "습니다" : "지 않습니다"); break;
		case 4: s1 = s2; break;								/* s2를 s1에 대입 */
		case 5: printf("s1 == s2 = %s\n", s1 == s2 ? "true" : "false");
			printf("s1 & s2 = "); PrintLn(s1 & s2);
			printf("s1 | s2 = "); PrintLn(s1 | s2);
			printf("s1 - s2 = "); PrintLn(s1 & ~s2);
			break;
		case 6: x = scan_data(ADD); Add(&s2, x); break;		/* 추가 */
		case 7: x = scan_data(RMV); Remove(&s2, x); break;	/* 삭제 */
		case 8: x = scan_data(SCH); idx = IsMember(s2, x);	/* 검색 */
			printf("s2에 들어있%s.\n", idx != 0 ? "습니다" : "지 않습니다"); break;
		case 9: s2 = s1; break;								/* s1을 s2에 대입 */
		}
	}
	
	return 0;
}